## 1 - Packages ##

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn.model_selection as sk

%matplotlib inline

In [3]:
# dir(sklearn.model_selection)
# help(sk.train_test_split)

## 2 - Load data, reshape and standardize ##

In [4]:
###For better readability
col_name = ["pregnant", "glucose", "bp", "skin", "insulin", "bmi", "pedigree", "age", "label"]
dataset = pd.read_csv("pima_indians_diabetes.csv", names = col_name)
# print(dataset)

In [6]:
###To read 5 rows of data
dataset.head()

,pregnant,glucose,bp,skin,insulin,bmi,pedigree,age,label
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [7]:
###Load data and labels
features = col_name[: len(col_name) - 1]
X = dataset[features]
y = dataset.label

In [8]:
###Split data into train and test set
X_train, X_test, y_train, y_test = sk.train_test_split(X, y, test_size = 0.25, random_state = 0)

# print(X_test.shape[0])
# print(X_train.shape)
# print(y_train.shape)
# print(y_test.shape)

In [9]:
###Convert pandas dataframe to numpy array
train_set_x_orig = X_train.to_numpy()
test_set_x_orig = X_test.to_numpy()
train_set_y = y_train.to_numpy()
test_set_y = y_test.to_numpy()

In [11]:
###Display shape of training and test examples
m_train = train_set_x_orig.shape[0]
m_test = test_set_x_orig.shape[0]
num_px = train_set_x_orig.shape[1]

print ("Number of training examples: m_train = " + str(m_train))
print ("Number of testing examples: m_test = " + str(m_test))
print ("Height/Width of each image: num_px = " + str(num_px))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_set_x shape: " + str(X_train.shape))
print ("train_set_y shape: " + str(y_train.shape))
print ("test_set_x shape: " + str(X_test.shape))
print ("test_set_y shape: " + str(y_test.shape))


Number of training examples: m_train = 576
Number of testing examples: m_test = 192
Height/Width of each image: num_px = 8
Each image is of size: (8, 8, 3)
train_set_x shape: (576, 8)
train_set_y shape: (576,)
test_set_x shape: (192, 8)
test_set_y shape: (192,)


In [12]:
# Reshape the training and test examples
train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0], -1).T
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0], -1).T

print ("train_set_x_flatten shape: " + str(train_set_x_flatten.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x_flatten shape: " + str(test_set_x_flatten.shape))
print ("test_set_y shape: " + str(test_set_y.shape))

train_set_x_flatten shape: (8, 576)
train_set_y shape: (576,)
test_set_x_flatten shape: (8, 192)
test_set_y shape: (192,)


In [13]:
###Standardize dataset
'''
mean_X_train = np.mean(train_set_x_flatten, axis = 0)
mean_X_test = np.mean(test_set_x_flatten, axis = 0)

train_set_x = train_set_x_flatten - mean_X_train
test_set_x = test_set_x_flatten - mean_X_test

std_X_train = np.std(train_set_x, axis = 0)
std_X_test = np.std(test_set_x, axis = 0)

train_set_x = train_set_x / std_X_train
test_set_x = test_set_x / std_X_test
print(train_set_x)
'''

train_set_x = (train_set_x_flatten - np.mean(train_set_x_flatten, axis = 0)) / np.std(train_set_x_flatten, axis = 0)
test_set_x = (test_set_x_flatten - np.mean(test_set_x_flatten, axis = 0)) / np.std(test_set_x_flatten, axis = 0)

## 3 - Building the parts of our algorithm ## 
### 3.1 - Helper functions

In [14]:
def sigmoid(z):
    '''
        OBJECTIVE: Compute the sigmoid of z.
        INPUT PARAMETERS: 
                        z : A scalar or numpy array of any size.
        RETURN VALUE:
                        s : sigmoid(z)
    '''

    s = 1 / (1 + np.exp(-z))
    
    return s

### 3.2 - Initializing parameters

In [15]:
def initialize_with_zeros(dim):
    
    '''
        OBJECTIVE: This function creates a vector of zeros of shape (dim, 1) for w and initializes b to 0.
        INPUR PARAMETERS:
                        dim : size of the w vector we want (or number of parameters in this case)
        RETURN VALUE:
                        w : initialized vector of shape (dim, 1)
                        b : initialized scalar (corresponds to the bias)
    '''
    
    w = np.zeros((dim, 1))
    b = 0
    
    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

### 3.3 - Forward and Backward propagation

In [23]:
def propagate(w, b, X, Y):
    
    '''
        OBJECTIVE: Implement the cost function and its gradient for the propagation.
        INPUT PARAMETERS:
                        w : weights, a numpy array of size (8, 1)
                        b : bias, a scalar
                        X : data of size (8, number of examples) #There are 64 features in the dataset.
                        Y : true "label" vector (1, number of examples)
        RETURN VALUE:
                        cost : negative log-likelihood cost for logistic regression
                        dw : gradient of the loss with respect to w, thus same shape as w
                        db : gradient of the loss with respect to b, thus same shape as b
    '''
    
    m = X.shape[1]
    
    # FORWARD PROPAGATION (FROM X TO COST)
    A = sigmoid(np.dot(w.T, X) + b)                                   # compute activation
    cost = (-1/m) * np.sum(Y * np.log(A) + (1 - Y) * np.log(1 - A), axis = 1, keepdims = True)             # compute cost
    
    # BACKWARD PROPAGATION (TO FIND GRAD)
    dw = (1 / m) * np.dot(X, (A - Y).T)
    db = (1 / m) * np.sum((A - Y), axis = 1, keepdims = True)

    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

### 3.4 - Optimization

In [24]:
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    '''
        OBJECTIVE: This function optimizes w and b by running a gradient descent algorithm
        INPUT PARAMETERS:
                        w : weights, a numpy array of size (8, 1)
                        b : bias, a scalar
                        X : data of shape (8, number of examples)
                        Y : true "label" vector of shape (1, number of examples)
                        num_iterations : number of iterations of the optimization loop
                        learning_rate : learning rate of the gradient descent update rule
                        print_cost : True to print the loss every 100 steps
    
        RETURN VALUE:
                        params : dictionary containing the weights w and bias b
                        grads : dictionary containing the gradients of the weights and bias with respect to the cost function
                        costs : list of all the costs computed during the optimization, this will be used to plot the learning curve.
    '''

    costs = []
    
    for i in range(num_iterations):
        
        
        # Cost and gradient calculation
        grads, cost = propagate(w, b, X, Y)
        
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # update rule
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        
        # Print the cost every 100 training iterations
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [25]:
def predict(w, b, X):
    '''
        OBJECTIVE: Predict whether the label is 0 or 1 using learned logistic regression parameters (w, b)
        INPUT PARAMETERS:
                        w : weights, a numpy array of size (8, 1)
                        b : bias, a scalar
                        X : data of size (8, number of examples)
    
        RETURN VALUE:
                        Y_prediction : a numpy array (vector) containing all predictions (0/1) for the examples in X
    '''
    
    m = X.shape[1]  #no. of training examples
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    
    # Compute vector "A" predicting the probabilities of a cat being present in the picture
    A = sigmoid(np.dot(w.T, X) + b)
    
    for i in range(A.shape[1]):
        
        # Convert probabilities A[0,i] to actual predictions p[0,i]
        Y_prediction[0, i] = np.where(A[0, i] > 0.4, 1, 0)
    
    assert(Y_prediction.shape == (1, m))
    
    return Y_prediction

## 4 - Merge all functions into a model ##

In [27]:
def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):
    '''
        OBJECTIVE: Builds the logistic regression model.
        INPUT PARAMETERS:
                        X_train : training set represented by a numpy array of shape (8, m_train)
                        Y_train : training labels represented by a numpy array (vector) of shape (1, m_train)
                        X_test : test set represented by a numpy array of shape (8, m_test)
                        Y_test : test labels represented by a numpy array (vector) of shape (1, m_test)
                        num_iterations : hyperparameter representing the number of iterations to optimize the parameters
                        learning_rate : hyperparameter representing the learning rate used in the update rule of optimize()
                        print_cost : Set to true to print the cost every 100 iterations
        RETRUN VALUE:
                        d : dictionary containing information about the model.
    '''
    
    # initialize parameters with zeros
    w, b = initialize_with_zeros(X_train.shape[0])

    # Gradient descent
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
    
    # Retrieve parameters w and b from dictionary "parameters"
    w = parameters["w"]
    b = parameters["b"]
    
    # Predict test/train set examples
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)

    # Print train/test Errors
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [29]:
d = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 1000, learning_rate = 0.005, print_cost = True)

Cost after iteration 0: 0.693147
Cost after iteration 100: 0.655231
Cost after iteration 200: 0.646350
Cost after iteration 300: 0.643395
Cost after iteration 400: 0.641784
Cost after iteration 500: 0.640543
Cost after iteration 600: 0.639446
Cost after iteration 700: 0.638438
Cost after iteration 800: 0.637498
Cost after iteration 900: 0.636618
train accuracy: 63.88888888888889 %
test accuracy: 67.70833333333333 %
